In [1]:
import numpy as np

from human_terms_network import Human_Terms_Network
from dataset_load import *
from lr_baseline import LRBaseline

Using TensorFlow backend.


In [2]:
# open_pickle in dataset_load

X_train_original, X_test_original, y_train_original, y_test_original = load_imdb('../../data/imdb')

In [3]:
word_list, connotation = load_unigrams('./imdb-unigrams.txt', X_train_original, y_train_original)

In [4]:
y_train_agreement, y_test_agreement = generate_appearance(X_train_original, X_test_original, 
                                                          word_list, connotation)

In [5]:
imdb_baseline = LRBaseline(X_train_original, X_test_original, y_train_original, y_test_original, 
                           human_terms=word_list)

In [6]:
print(imdb_baseline.baseline())
print(imdb_baseline.baseline(penalty='l1'))
print(imdb_baseline.human_terms_baseline())
print(imdb_baseline.human_terms_baseline(penalty='l1'))

(0.94536, 0.86476, 0.36659306060025315)
(0.9424, 0.87184, 0.3358980862841266)
(0.81476, 0.834936821103968, 0.3871934758932064, 0.0978)
(0.81452, 0.8347594768344048, 0.3873009491304477, 0.0978)


In [7]:
X_train = imdb_baseline.human_terms_cv.transform(X_train_original)
X_test = imdb_baseline.human_terms_cv.transform(X_test_original)
y_train = y_train_original
y_test = y_test_original

In [8]:
indices = np.where(np.sum(X_train,axis=1)!=0)[0]
X_test_nreject = X_test[indices]
y_test_nreject = y_test[indices]

In [9]:
y_test_nreject.shape

(22614,)

In [10]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=43, penalty='l1')
clf.fit(X_train, y_train)

print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))

print(clf.score(X_test_nreject, y_test_nreject))
print((X_test.shape[0] - len(indices))/X_test.shape[0])
print(X_test.shape[0] - len(indices))

0.81472
0.81068
0.8109135933492527
0.09544
2386


In [11]:
X_train = imdb_baseline.baseline_cv.transform(X_train_original)
X_test = imdb_baseline.baseline_cv.transform(X_test_original)
y_train = y_train_original
y_test = y_test_original

In [12]:
X_test[10]

<1x3686 sparse matrix of type '<class 'numpy.int64'>'
	with 224 stored elements in Compressed Sparse Row format>

In [13]:
imdb_htm = Human_Terms_Network(input_shape=X_train.shape[1],
                               human_terms_shape=len(word_list),
                              domain='imdb')

In [14]:
# imdb_htm.combined.load_weights('./combined/combined-weight-improvement-10-0.34.hdf5')

In [15]:
imdb_htm.set_data(X_train, X_test, y_train_agreement, y_test_agreement, y_train, y_test)

In [16]:
# imdb_htm.train(epochs=1, save_model=False, show_graph=False)

In [17]:
imdb_htm.test(reject=True)

# 0.4 with validation_split = 0.33

RuntimeError: You must compile a model before training/testing. Use `model.compile(optimizer, loss)`.

In [ ]:
y_test_tanh = y_test
y_test_tanh[y_test_tanh == 0] = -1
y_test_tanh

In [ ]:
X_test.shape

In [ ]:
imdb_htm.base_combined.evaluate(X_test, y_test_tanh)

In [ ]:
imdb_htm.combined.evaluate([X_test, y_test_agreement], y_test)

In [ ]:
imdb_htm.combined.evaluate([X_train, y_train_agreement], y_train)

In [ ]:
imdb_htm.test(reject=False)

In [ ]:
from keras.models import Model

human_terms_relu_model = Model(inputs=imdb_htm.combined.input,
                                            outputs=imdb_htm.combined.get_layer('concatenate').output)
predict_relu = human_terms_relu_model.predict([X_test, y_test_agreement])
accept_indices = np.where(np.sum(predict_relu, axis=1)!=0)
total_reject = X_test.shape[0] - len(accept_indices)
rejection_rate = total_reject/X_test.shape[0]

In [ ]:
rejection_rate

In [ ]:
eval = imdb_htm.combined.evaluate([imdb_htm.X_test, imdb_htm.y_test_agreement], imdb_htm.y_test)

In [ ]:
eval